Preprocessing 🤖
---

In [10]:
import pandas as pd
import numpy as np
import json

In [7]:
# import data
data_for_preprocessing_path = '../merged_data/data_for_preprocessing.csv'
df = pd.read_csv(data_for_preprocessing_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386457 entries, 0 to 386456
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Unnamed: 0          386457 non-null  int64 
 1   lang                386457 non-null  object
 2   possibly_sensitive  386457 non-null  object
 3   source              386457 non-null  object
 4   text                386457 non-null  object
 5   created_at          386457 non-null  object
 6   location            299526 non-null  object
 7   win                 386457 non-null  object
 8   is_poke             386457 non-null  object
 9   broad_location      386408 non-null  object
 10  hashtags            386457 non-null  object
 11  has_hashtags        386457 non-null  bool  
dtypes: bool(1), int64(1), object(10)
memory usage: 32.8+ MB


In [14]:
# import dictionaries
hashtag_to_game_path = '../EDA/hashtag_to_game.json'
with open(hashtag_to_game_path, 'r') as this_file:
    hashtag_to_game = json.loads(this_file.read())

locs_path = '../EDA/loc_dict.json'
with open(locs_path, 'r') as file:
    locs = json.loads(file.read())

In [38]:
len(set(locs.values()))

80

# Create columns for lang

Use the 

In [31]:
df['lang'].value_counts()[:25]

en           310370
ja            38369
es            10390
tr             5690
und            5160
undefined      3558
fr             2629
ko             2197
de             1910
ca             1016
ru              983
et              775
th              489
tl              449
nl              328
cs              264
pt              237
in              207
zh              202
fi              147
no              144
ro              138
it              131
ar               67
da               59
Name: lang, dtype: int64